In [8]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import h5py
import PIL
import seaborn as sns
import plotly
import sklearn_pandas
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.preprocessing import MinMaxScaler

In [9]:
# Esta celda lo que hace es obtener el dataset que se encuentra en el repositorio, utilizando un path relativo
path = '..//TP2//Seasons_Stats.csv'
data = pd.read_csv(path)

In [10]:
# Con esta celda buscamos reducir la cantidad de columnas que posee el dataset, eliminando las columnas que pensamos no son necesarias para realizar la predicción.
columns = ['blanl','blank2', 'Tm']
data = data.drop(columns, axis=1)

In [11]:
# Eliminamos todas las filas de datos que sean anteriores al año 1980. Elegimos este año ya que a partir de aquí se comienzan a tener en cuenta estadísticas como, por ejemplo, los tiros de 3 puntos.
# Esto lo hacemos ya que, antes de ese año, el dataset poseía muchas estadísticas que todavía no se tenian en cuenta y había muchos valores en NaN
data = data.drop(data[data.Year < 1980].index, axis=0)
# Además, eliminamos todos los datos en NaN para evitar inconsistencias en el dataset.
#data = data.dropna()

#Con esta linea mostramos todas las columnas del dataset
pd.set_option('display.max_columns', 100)

In [12]:
df = pd.DataFrame(data)

In [13]:
df[['3P%','FT%', '2P%', '3PAr', 'FTr', 'FG%', 'eFG%', 'TS%', 'TOV%']] = df[['3P%','FT%', '2P%', '3PAr', 'FTr', 'FG%', 'eFG%', 'TS%', 'TOV%']].fillna(0)

In [14]:
# Este celda elimina los años que figuran mas de una vez para un mismo jugador.

df = df.drop_duplicates(subset=['Player','Year'])

In [15]:
# Esta celda renombra la primer columna del dataFrame (denominada 'Unnamed: 0') y le setea el nombre de "id"
df = df.rename(columns = {'Unnamed: 0' : 'id'})

In [16]:
# Con esta celda, creamos la que será nuestra columna TARGET y la llamamos "morePoints" ("Mas Puntos")
# Dicha columna podrá tener solo dos valores "Yes"/"No" dependiendo si el jugador hizo mas puntos que la temporada anterior
# Cabe recalcar que si es la 1er temporada del jugador o si una temporada no jugo por algun motivo, se colocará en "Yes" ya que consideramos que al no haber jugado efectivamente realizó mas puntos.
df.insert(18, 'morePoints', 'No', allow_duplicates = True)

In [17]:
# Esta es una funcion que realizamos para setear la columna "morePoints"
# Buscamos el año anterior y comparamos los puntos que realizó en ese año y el año actual.
# Si efectivamente hizo mas puntos no hacemos nada, pero si no seteamos la columna en "No"
def funcionSetMorePoints(id):
    auxiliar = df.loc[id]
    temporadaSiguiente = df.loc[(df.Player == auxiliar.Player) & (df.Year == auxiliar.Year  + 1)]      
    if ((not temporadaSiguiente.empty) and (auxiliar.PTS <= temporadaSiguiente.PTS.item())):
        df.loc[df.id == id,'morePoints'] = 'Yes'
    else:
        if temporadaSiguiente.empty:
            df.loc[df.id == id,'morePoints'] = 'None'
    return id

In [18]:
# Llamamos a la funcion defenida anteriormente
df.id = df.id.apply(funcionSetMorePoints)

In [ ]:
df = df[df['morePoints'] != 'None']
display(df)

# TP N° 2 - Experimentación

## Métrica a utilizar

Accuracy? <br>
Problemas de Accuracy:<br>
•	Desbalances en la data (dos clases de diferente crucialidad o peso son ponderadas de la misma forma).<br>
•	Si erra por sí o por no, tiene diferente impacto. Es mejor cuando erra algo que resulta positivo, por ejemplo, que prediga que hay cáncer cuando en realidad no lo había.

**No tenemos ninguno de estos dos problemas, para mi podría ser Accuracy ANASHEEEIIII.**

## Técnica de feature engineering

???? No tengo idea xd. 

Podría ser Binning (para achicar los decimales).